### Diving into Pinecone

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
import pinecone
pinecone.init(
    api_key=os.environ.get("PINECONE_API_KEY"),
    environment=os.environ.get("PINECONE_ENV")
)

C:\Users\Younes\AppData\Roaming\Python\Python39\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
pinecone.info.version()

VersionResponse(server='2.0.11', client='2.2.2')

### Pinecone Indexes

In [4]:
pinecone.list_indexes()

[]

In [9]:
index_name = 'langchain-pinecone'
if index_name not in pinecone.list_indexes():
    print(f'Creating index {index_name}...')
    pinecone.create_index(index_name, dimension=1536, metric='cosine', pods=1, pod_type='p1.x2')
    print('Done')
else:
    print(f'Index {index_name} already exists!')

Creating index langchain-pinecone...
Done


In [7]:
pinecone.describe_index(index_name)

IndexDescription(name='langchain-pinecone', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [8]:
# deleting an index
index_name = 'langchain-pinecone'
if index_name in pinecone.list_indexes():
    print(f"Deleting index {index_name}...")
    pinecone.delete_index(index_name)
    print('Done')
else:
    print(f'Index {index_name} does not exists!')

Deleting index langchain-pinecone...
Done


In [10]:
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [14]:
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
vectors
ids = list("abcde")

In [27]:
index_name = 'langchain-pinecone'
index = pinecone.Index(index_name)
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [18]:
# updating a vector
index.upsert(vectors=[("c", [0.3]*1536)])

{'upserted_count': 1}

In [19]:
# fetching a vector
index = pinecone.Index('langchain-pinecone')
index.fetch(ids=['c', 'd'])

{'namespace': '',
 'vectors': {'c': {'id': 'c',
                   'values': [0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
                              0.3,
       

In [20]:
# deleting vectors
index.delete(ids=['b', 'c'])

{}

In [25]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [23]:
index.fetch(ids=['b'])

{'namespace': '', 'vectors': {}}

In [24]:
index.delete(delete_all=True)

{}

In [29]:
# querying

queries = [[random.random() for _ in range(1536)] for v in range(2)]

In [30]:
index.query(
    queries=queries,
    top_k=3,
    include_values=False
)

{'results': [{'matches': [{'id': 'c', 'score': 0.750601411, 'values': []},
                          {'id': 'd', 'score': 0.747686565, 'values': []},
                          {'id': 'e', 'score': 0.740642667, 'values': []}],
              'namespace': ''},
             {'matches': [{'id': 'b', 'score': 0.763228, 'values': []},
                          {'id': 'c', 'score': 0.76265949, 'values': []},
                          {'id': 'e', 'score': 0.761710882, 'values': []}],
              'namespace': ''}]}